In [134]:
import pandas as pd
from Librerias_SAP import SAP_GUI, Funtions
import re
from datetime import datetime, timedelta

In [135]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
#Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [136]:
# Disposiciones ZSD110
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar

default_column_Canal01_Pendientes=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Pedido', 'Clase Pedi', 'Creado el', 'Pedido.1', 'Posición', 'Material',
       'Vlr.Neto P', 'Moneda', 'Cantidad P', 'Cantidad C', 'UM venta',
       'Vlr.Neto C', 'Status Glo', 'Status Tot', 'Status Ent', 'Status Ent.1',
       'Status Ent.2']  

default_column_Canal01_Entregado=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT']

default_column_Pedidos=['OrgVt', 'CDis', 'BqEn', 'Solic.', 'PrimFeEntr', 'ÚltEntrega',
       'FePrefEnt.', 'ClVt', 'Valor neto', 'Mon.', 'Doc.venta', 'ST',
       'Solicitante', 'Nº pedido cliente', 'Descrip.breve', 'Nº de cliente 1',
       'Contador de pedidos', 'Dirección', 'Barrio', 'Población', 'Teléfono']

Columns_Data_ZSD10_Exito_Cencosub_ZVMI=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Columns_Data_ZSD038_Exito_Cencosub_Rechazados=['Doc.comer.', 'Fecha doc.', 'Destinat.', 'Nombre 1', 'Cód.Vend.',
       'Nombre vendedor', 'MR', 'Denominación', 'Material', 'Denominación.1',
       'Marca', 'Categoría', 'Ctd.confirmada', 'Valor neto', 'Mon.', 'CDis',
       'Solic.', 'Nombre']

Columns_Data_ZSD10=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Column_Data_ZSD76=['Solic.', 'Solicitante', 'Fecha salida mercancía', 'Material',
       'Número de material', 'Cantidad entrega', 'UM', 'MARCA', 'CATEGORIA',
       'Entrega', 'ClEnt', 'Clase de entrega', 'ClVt',
       'Clase de documento de ventas', 'OrgVt', 'CDis', 'Doc.venta',
       'Prc.neto', 'Mon.', 'Destinat.', 'Destinatario de mercancías',
       'Nº pedido cliente', 'Fecha creación Pedido', 'Cont.']

Column_Data_ZSD038=['Doc.comer.', 'Fecha doc.', 'Destinat.', 'Nombre 1', 'Cód.Vend.',
       'Nombre vendedor', 'MR', 'Denominación', 'Material', 'Denominación.1',
       'Marca', 'Categoría', 'Ctd.confirmada', 'Valor neto', 'Mon.', 'CDis',
       'Solic.', 'Nombre']

"""----------------------------------------------LEER ARCHIVOS--------------------------------"""
#Data Pendientes
Canal01_Pendiente=pd.read_csv("C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Funtions.Clean_Columns(Canal01_Pendiente)
Canal01_Pendiente=Funtions.default_column(default_column_Canal01_Pendientes,Canal01_Pendiente)

#Data Entregados
Canal01_Entregado=pd.read_csv("C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Funtions.Clean_Columns(Canal01_Entregado)
Canal01_Entregado=Funtions.default_column(default_column_Canal01_Entregado,Canal01_Entregado)

"""""-------------------------------------------Limpio los datos-----------------------------------"""

# Esta session combierte colunmas de float .0 a enteros int, tambien recibe valores Null
Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1']=Funtions.Estandarizo_Pedidos(Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1'])


In [137]:
"""Seccion para coompletar los valores necesarios para trabajar"""

Agenda=["85","20","146","149","50","138","45"]
# En este caso solo aplica para Entrega y pendiente en el cual completa con 00 los pedidos del Exito
Canal01_Entregado['Pedido.1']=Funtions.complete_pedidos(Canal01_Entregado['Pedido.1'],Agenda)
Canal01_Pendiente['Pedido.1']=Funtions.complete_pedidos(Canal01_Pendiente['Pedido.1'],Agenda)

In [138]:
"""---------------------------------------Unifico los dataframes de pedido y entregas--------------------------------------------------------------"""
Default_Column_Informe=['Canal','Denomin.','Solic.','Nombre','Pedido','Clase Pedido','Material','Valor Neto','Cantidad']

longuitud_Pendiente=len(Canal01_Pendiente)
longuitud_Entrega=len(Canal01_Entregado)

Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]

Canal01_Pendiente_Informe=Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado_Informe=Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]

print(longuitud_Pendiente,longuitud_Entrega)

"""---En esta sección analizar si borro duplicados-----"""
Canal01_Entregado_Pendiente_Informe=pd.concat([Funtions.default_column(Default_Column_Informe,Canal01_Pendiente_Informe),Funtions.default_column(Default_Column_Informe,Canal01_Entregado_Informe)]).drop_duplicates().reset_index(drop=True)

3720 869


In [139]:
"""------------------------Optener datos de Agenda_Exito y Cencosub------------------"""
#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["93","122","127","95","60"]


"""-----------------------------Aplico filtro en Agenda exito Entrega----------------------------------"""
Filtro_Agenda_Exito_Entrega_Pendiente=Funtions.Search_Agenda_Exito(Canal01_Entregado_Pendiente_Informe['Pedido'],Agenda_Exito)
Filtro_Agenda_Cencosub_Entrega_Pendiente=Funtions.Search_agenda_Cencosub(Canal01_Entregado_Pendiente_Informe["Pedido"],Agenda_Cencosub)
Filtro_Sin_Agenda_Cencosub_Entrega_Pendiente=Filtro_Agenda_Exito_Entrega_Pendiente+Filtro_Agenda_Cencosub_Entrega_Pendiente

In [140]:
"""-------------------------------------------------Aplico filtro para el consolidados------------------------------------------------------------"""


"""------------------------------Canal 01 Entrega con agenda Exito-----------------------"""
#Agenda Exito
Filtro_Canal01_Entregado_Pendiente_Exito=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Exito_Entrega_Pendiente))   #Excluyo con ~

#Agenda Censosud
"""------------------------------Canal 01 Entrega con agenda Cencosub-----------------------"""
Filtro_Canal01_Entregado_Pendiente_Cencosub=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Cencosub_Entrega_Pendiente))   #Excluyo con ~

In [141]:
"""----------------------------------------------Creao tabla dinamica general--------------------------------------------------------------------------------"""
Tabla_dinamica_Canal01_Entregado_Pendiente_Informe=Canal01_Entregado_Pendiente_Informe.pivot_table(index=['Pedido','Clase Pedido'],aggfunc={'Material':'count','Cantidad':sum,'Valor Neto':sum}).reset_index()

In [142]:
# Tabla dinácamica Exito
Table_dinamica_Exito_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Exito_Entrega_Pendiente)]
# Tabla dinácamica Exito Cencosud
Table_dinamica_Cencosub_entrega_pendiente=Tabla_dinamica_Canal01_Entregado_Pendiente_Informe[Tabla_dinamica_Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Cencosub_Entrega_Pendiente)]

### Realizo primera busquedad para fechas de la transsaccion ZSD037

In [143]:
# Copio todos los pedidos de la agenda Exito y Cencosub
pedidos_Exito_Cencosub=pd.concat([Table_dinamica_Exito_entrega_pendiente['Pedido'],Table_dinamica_Cencosub_entrega_pendiente['Pedido']])


In [144]:
#Busco en la tansaccion ZSD037 los pedidos de las tablas dinamicas tanto de Exito y Cencosub

Transsaccion_ZSD037="zsd037"
SAP_GUI.Search_Pedidos_ZSD037(Transsaccion_ZSD037,pedidos_Exito_Cencosub,session)

In [145]:
# Guardar archivo que contiene la fecha de los pedidos Exito y Cencosud
Name_ZSD037="Pedidos pendientes Exito y Cencosub"
Ruta_ZSD037_Exito_Cencosub="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD037,session,Ruta_ZSD037_Exito_Cencosub)

In [146]:
# Utilizo la transaccion para buscar ZSD037 en entrega para las fechas
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD037_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Data_ZSD037_Pedidos_Exito_Cencosub[Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]

In [147]:
#Estandarizo los pedidos del Exito

# Puede que se represente inconsistencia entre str y int por lo cual estandarizo pedidos
try:
    Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"]=Funtions.Clean_int_to_str(Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"])
    Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"]=Funtions.complete_pedidos(Data_ZSD037_Pedidos_Exito_Cencosub["Nº pedido cliente"],Agenda)
except:
    pass


### Segunda busqueda pedidos de facturación día anterior en la transaccion ZSD035D

In [148]:
transsaccion="ZSD035D"
variant="JESPINOSAP"
provision="INFORME FACTU"

In [149]:
# Me ubico un día atras segun el calendario
Un_dia_atras=(datetime.now()-timedelta(days=1)).date()
Un_dia_atras=Funtions.stand_day(Un_dia_atras)

# Busco en la transaccion ZSD035D
SAP_GUI.Search_ZSD035D(transsaccion,session,provision,Un_dia_atras,variant)

Name_ZSD035D="Pendiente_Clientes"
Ruta_ZSD035D="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"

SAP_GUI.Export_TXT2(Name_ZSD035D,session,Ruta_ZSD035D)

In [150]:
# Leer el archivo del informe de facturación ZSD035D
url_ZSD035D="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"+str(Name_ZSD035D)+".txt"
Data_ZSD035D=pd.read_csv(url_ZSD035D,skiprows=5,delimiter="\t")
Data_ZSD035D=Funtions.Clean_Columns(Data_ZSD035D)
Data_ZSD035D=Data_ZSD035D[Data_ZSD035D['Material'].notnull()]   #Elimino valor nulo de la ultima fila

try:
    #Tener encuenta esta lineas puede este en dato numerico
    Data_ZSD035D["Lineas"]=Data_ZSD035D["Lineas"].apply(lambda x:Funtions.Clean_num(x))    
except:
    pass


In [151]:
Table_dinamica_Cencosub_entrega_pendiente

,Pedido,Clase Pedido,Cantidad,Material,Valor Neto
24,122-444641,ZEDI,12,1,"272,052"
25,127-448885,ZEDI,12,1,"272,052"
26,127-448893,ZEDI,12,1,"272,052"
39,93-490535,ZEDI,4,1,"138,476"
40,93-490639,ZEDI,12,1,"272,052"
41,93-490644,ZEDI,12,1,"272,052"
42,95-498897,ZEDI,12,1,"272,052"
43,95-498907,ZEDI,12,1,"272,052"


In [152]:
# Debo de filtrar por ZVMI y exito y cencusub del día anterior  

"""Tendria que usar el excel de sharepoint como sugerencia"""

filtro_ZVMI_Exito=Table_dinamica_Exito_entrega_pendiente["Clase Pedido"]=="ZVMI"
filtro_ZVMI_Cencosub=Table_dinamica_Cencosub_entrega_pendiente["Clase Pedido"]=="ZVMI"

Tabla_Agenda_Exito_ZVMI=Table_dinamica_Exito_entrega_pendiente.loc[filtro_ZVMI_Exito]
Tabla_Agenda_Cencosub_ZVMI=Table_dinamica_Cencosub_entrega_pendiente.loc[filtro_ZVMI_Cencosub]

In [153]:
Tabla_Agenda_Exito_Cencosub_ZVMI=pd.concat([Tabla_Agenda_Exito_ZVMI,Tabla_Agenda_Cencosub_ZVMI],axis=0)



In [154]:
Tabla_Agenda_Exito_Cencosub_ZVMI

,Pedido,Clase Pedido,Cantidad,Material,Valor Neto
5,0085782604,ZVMI,1324,690,"5,040 39,328 13,102 8,000 14..."
6,0085782605,ZVMI,492,230,"116,772 10,080 20,773 43,630 53..."
7,0085782606,ZVMI,1,1,"13,109"
8,0085782607,ZVMI,4,3,"12,437 11,428 5,714"
10,0085782643,ZVMI,4,2,"13,107 39,321"
11,0085782644,ZVMI,21,10,"9,412 36,928 26,218 18,464 52..."
12,0085782645,ZVMI,5,2,"26,214 39,321"
13,0085782646,ZVMI,20,14,"18,824 15,798 9,412 65,884 55..."
16,0085782658,ZVMI,1,1,"21,445"
20,0085782771,ZVMI,1,1,"12,437"


### Paso utilizo transsaccion ZSD10

In [155]:
transsaccion_ZSD10="ZSD10"
provision_ZSD10="PEDIDOS"
variant_ZSD10="JESPINOSAP"


def Search_ZSD10(transsaccion,Series,variant,provision,session):
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    variant: Variante a buscar
    provision: disposicion de interes
    session: session activa
    """
    session.StartTransaction(transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SP$00013_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        tabla=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(tabla)
    except:
        return(session)

#Busco en la correpondiente transaccion ZSD10

Search_ZSD10(transsaccion_ZSD10,Tabla_Agenda_Exito_Cencosub_ZVMI["Pedido"],variant_ZSD10,provision_ZSD10,session)


<COMObject <unknown>>

In [156]:
# Descargo transsación transaccion ZSD10
Name_ZSD010_Exito_Cencosub_ZVMI="Exito_Cencosub_ZVMI"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD010_Exito_Cencosub_ZVMI,session,Ruta_Exito_ZSD010_ZVMI)

In [157]:
# Exportar de la transsaccion ZSD10 exporta los archivos

url_ZSD10_Exito_Cencosub_ZVMI=Ruta_Exito_ZSD010_ZVMI+Name_ZSD010_Exito_Cencosub_ZVMI+".txt"
Data_ZSD10_Exito_Cencosub_ZVMI=pd.read_csv(url_ZSD10_Exito_Cencosub_ZVMI,skiprows=5,delimiter="\t")
Data_ZSD10_Exito_Cencosub_ZVMI=Funtions.Clean_Columns(Data_ZSD10_Exito_Cencosub_ZVMI)
Data_ZSD10_Exito_Cencosub_ZVMI=Funtions.default_column(Columns_Data_ZSD10_Exito_Cencosub_ZVMI,Data_ZSD10_Exito_Cencosub_ZVMI)
Data_ZSD10_Exito_Cencosub_ZVMI=Data_ZSD10_Exito_Cencosub_ZVMI[Data_ZSD10_Exito_Cencosub_ZVMI["Cliente"].notnull()]

# Transformo datos 
Data_ZSD10_Exito_Cencosub_ZVMI["Unidades"],Data_ZSD10_Exito_Cencosub_ZVMI["ValorNeto"],Data_ZSD10_Exito_Cencosub_ZVMI["Total Pedidos"]=Funtions.Clean_Num_List(Data_ZSD10_Exito_Cencosub_ZVMI["Unidades"],Data_ZSD10_Exito_Cencosub_ZVMI["ValorNeto"],Data_ZSD10_Exito_Cencosub_ZVMI["Total Pedidos"])


# Tranformo la columna que tiene .0 por si es necesario
try:
    # La columna Doc.venta es importante ya que se lo utiliza en la suiente transacción
    Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta']=Funtions.Clean_int_to_str(Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta'])
except:
    None

In [158]:
Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta']



0       903156632
1       903156641
2       903156641
3       903156641
4       903156653
          ...    
4620    903157158
4621    903157158
4622    903157158
4623    903157158
4624    903157158
Name: Doc.venta, Length: 4625, dtype: object

In [159]:
transsaccion_ZSD038="ZSD038"

In [160]:
# session.findById("wnd[0]/usr/btn%_S_VBELN_%_APP_%-VALU_PUSH").press()

def Search_ZSD038(transsaccion,Series,session):
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    variant: Variante a buscar
    provision: disposicion de interes
    session: session activa
    """
    session.StartTransaction(transsaccion)
    # session.findById("wnd[0]/tbar[1]/btn[17]").press()
    # session.findById("wnd[1]/usr/txtV-LOW").text = provision
    # session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    # session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    # session.findById("wnd[1]/tbar[0]/btn[8]").press()
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_S_VBELN_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        tabla=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(tabla)
    except:
        return(session)
    
Search_ZSD038(transsaccion_ZSD038,Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta'],session)

Name_ZSD038="Pedidos_Rechazados"
Ruta_ZSD038="C:\\Users\\practicante.picking\\OneDrive - Prebel S.A\\Escritorio\\bot_picking\\Archivos_CSV\\"

SAP_GUI.Export_TXT2(Name_ZSD038,session,Ruta_ZSD038)

In [164]:
url_ZSD038_Pedidos_Rechazados=Ruta_ZSD038+Name_ZSD038+".txt"
Data_ZSD038=pd.read_csv(url_ZSD038_Pedidos_Rechazados,skiprows=1,delimiter="\t")
Data_ZSD038=Funtions.Clean_Columns(Data_ZSD038)
Data_ZSD038=Funtions.default_column(Column_Data_ZSD038,Data_ZSD038)
Data_ZSD038=Data_ZSD038[Data_ZSD038["Material"].notnull()]

In [165]:
Data_ZSD038

,Doc.comer.,Fecha doc.,Destinat.,Nombre 1,Cód.Vend.,Nombre vendedor,MR,Denominación,Material,Denominación.1,Marca,Categoría,Ctd.confirmada,Valor neto,Mon.,CDis,Solic.,Nombre
0,903157160.0,19.07.2023,1.000000e+09,EXITO BUCARAMANGA LA ROSITA,102356.0,RV-YESSICA PAOLA VALERO JAIMES,04,Descontinuado,PB0084867,ESMALTE VITÚ NIEVE FREE 8ML,Vitú,Maquillaje,1.0,"2,823",COP,1.0,1.000000e+09,ALMACENES EXITO SA
1,903157160.0,19.07.2023,1.000000e+09,EXITO BUCARAMANGA LA ROSITA,102356.0,RV-YESSICA PAOLA VALERO JAIMES,Z4,Agotados Cierre Pedidos,PB0060789,BASE LASTING PERF MF 101 IVORY BGE,Max Factor,Maquillaje,6.0,"306,150",COP,1.0,1.000000e+09,ALMACENES EXITO SA
2,903157160.0,19.07.2023,1.000000e+09,EXITO BUCARAMANGA LA ROSITA,102356.0,RV-YESSICA PAOLA VALERO JAIMES,Z4,Agotados Cierre Pedidos,PB0064480,CORREC. CATR. LIQUID CAMO. BEIGE 5ML,Catrice,Maquillaje,1.0,"15,396",COP,1.0,1.000000e+09,ALMACENES EXITO SA
3,903157160.0,19.07.2023,1.000000e+09,EXITO BUCARAMANGA LA ROSITA,102356.0,RV-YESSICA PAOLA VALERO JAIMES,Z4,Agotados Cierre Pedidos,PB0069317,CORRECTOR VITÚ COLÁGENO SOYA 4 CASTAÑA,Vitú,Maquillaje,3.0,"31,263",COP,1.0,1.000000e+09,ALMACENES EXITO SA
4,903157160.0,19.07.2023,1.000000e+09,EXITO BUCARAMANGA LA ROSITA,102356.0,RV-YESSICA PAOLA VALERO JAIMES,Z4,Agotados Cierre Pedidos,PB0086403,CATR. BASE HIDRAT. TRUE SKIN TN010 30ML,Catrice,Maquillaje,1.0,"31,529",COP,1.0,1.000000e+09,ALMACENES EXITO SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,903157136.0,19.07.2023,1.000021e+09,ÉXITO PASEO DEL PUENTE,102356.0,RV-YESSICA PAOLA VALERO JAIMES,Z4,Agotados Cierre Pedidos,PB0037228,VITÚ BRILLO MANT KARITÉ 1 ARÁNDANO NI,Vitú,Maquillaje,3.0,"24,000",COP,1.0,1.000000e+09,ALMACENES EXITO SA
362,903156642.0,19.07.2023,1.000046e+09,ÉXITO LA DORADA,102352.0,RV - SOR MAYERLY CALEÑO VELASQUEZ,11,Destinatario no maneja material,PB0037201,VITU SUERO ANTIEDAD OJOS,Vitú,Tratamiento facial,3.0,"49,692",COP,1.0,1.000000e+09,ALMACENES EXITO SA
363,903156642.0,19.07.2023,1.000046e+09,ÉXITO LA DORADA,102352.0,RV - SOR MAYERLY CALEÑO VELASQUEZ,11,Destinatario no maneja material,PB0060329,PROM SPLASH VITU+ MINI SPLASH FELICITY,Vitú,Baño y Cuerpo,3.0,"68,364",COP,1.0,1.000000e+09,ALMACENES EXITO SA
364,903156642.0,19.07.2023,1.000046e+09,ÉXITO LA DORADA,102352.0,RV - SOR MAYERLY CALEÑO VELASQUEZ,11,Destinatario no maneja material,PB0060330,PROM SPLASH VITU+ MINI SPLASH DULCET,Vitú,Baño y Cuerpo,3.0,"68,364",COP,1.0,1.000000e+09,ALMACENES EXITO SA
